# VDS 데이터 웹크롤링

## 공공데이터 포털 웹크롤링

In [1]:
import requests
from requests_toolbelt import MultipartEncoder
import gzip
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
def post_vds(date):
    multipart = MultipartEncoder(fields={
        'dataSupplyDate': f'{date}',
        'dataSupplyYear': 'null',
        'dataSupplyMonth': 'null',
        'dataSupplyYearQ': 'null',
        'dataSupplyQuater': 'null',
        'dataSupplyYearY': 'null',
        'collectType': 'VDS',
        'dataType': '16',
        'collectCycle': '04',
        'supplyCycle': '01',
        'outFileName': f'vds_{date}.gz',
    })

    headers = {
        'Content-Type': multipart.content_type,
    }

    res = requests.post('http://data.ex.co.kr/portal/fdwn/log', headers=headers, data=multipart)

    return res.content

In [3]:
date = '20220701'

In [4]:
print(f"Start crawling {date}")

print("POST ...")
data = post_vds(f'{date}')
print("Decompressing ...")
decompressed_data = gzip.decompress(data)
print("Saving ...")
with open(f'vds_{date}.csv', 'wb') as w:
    w.write(decompressed_data)

print("Complete !")

Start crawling 20220701
POST ...
Decompressing ...
Saving ...
Complete !


In [5]:
df = pd.read_csv(f'vds_{date}.csv', encoding='euc-kr')
df

,수집일자,수집시분초,VDS_ID,지점이정,도로이정,점유율,평균속도,교통량,차로번호,콘존ID,콘존명,콘존길이,Unnamed: 12
0,20220701,0,0010VDE00100,1.40,0.20,0.0,0.0,0.0,1,0010CZE010,구서IC-영락IC,1.82,NaN
1,20220701,0,0010VDE00100,1.40,0.20,1.0,126.0,1.0,2,0010CZE010,구서IC-영락IC,1.82,NaN
2,20220701,0,0010VDE00100,1.40,0.20,0.0,0.0,0.0,3,0010CZE010,구서IC-영락IC,1.82,NaN
3,20220701,0,0010VDE00200,2.40,2.02,0.0,0.0,0.0,1,0010CZE011,영락IC-부산TG,1.99,NaN
4,20220701,0,0010VDE00200,2.40,2.02,1.0,77.0,2.0,2,0010CZE011,영락IC-부산TG,1.99,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41483503,20220701,235930,6000VDE03150,38.59,37.88,-1.0,-1.0,-1.0,2,6000CZE070,노포JC-금정IC,1.10,NaN
41483504,20220701,235930,6000VDS00450,6.23,9.04,-1.0,-1.0,-1.0,1,6000CZS020,한림IC-진영IC,3.85,NaN
41483505,20220701,235930,6000VDS00450,6.23,9.04,-1.0,-1.0,-1.0,2,6000CZS020,한림IC-진영IC,3.85,NaN
41483506,20220701,235930,6000VDS03150,38.59,38.98,-1.0,-1.0,-1.0,1,6000CZS070,금정IC-노포JC,1.10,NaN


## 데이터 확인

### 1. 점유율

In [6]:
df['점유율'].describe()

count    4.148351e+07
mean     3.315154e+00
std      5.805652e+00
min     -1.000000e+00
25%      0.000000e+00
50%      1.760000e+00
75%      4.810000e+00
max      1.000000e+02
Name: 점유율, dtype: float64

- 점유율 데이터가 존재하지 않으면 -1 값이 채워짐을 확인
- 비율 데이터로, 최소 0 최대 100의 데이터를 가지는 것을 확인
- 한양대 측에서 **aggregation할 때 산술평균으로 진행하면 된다고 했음**

### 2. 평균속도

In [7]:
df['평균속도'].describe()

count    4.148351e+07
mean     6.413794e+01
std      4.730450e+01
min     -1.000000e+00
25%      0.000000e+00
50%      8.400000e+01
75%      1.000000e+02
max      1.800000e+02
Name: 평균속도, dtype: float64

- 평균속도 또한 데이터가 존재하지 않으면 -1 값이 채워짐을 확인 (음수인 경우 데이터 제외하면 될 듯)
- 최고 속도가 180km/h로 비정상적인 데이터는 확인되지 않음
- 한양대 측에서 aggregation할 때 산술평균으로 진행하면 된다고 했음

### 3. 교통량

In [8]:
df['교통량'].describe()

count    4.148351e+07
mean     3.062110e+00
std      3.848199e+00
min     -1.000000e+00
25%      0.000000e+00
50%      2.000000e+00
75%      5.000000e+00
max      3.000000e+01
Name: 교통량, dtype: float64

- 교통량도 데이터가 존재하지 않으면 음수 데이터가 들어감을 확인
- 한양대 측에서 aggregation할 때 단순 합하면 된다고 했음

## 유효하지 않은 데이터 삭제
- 교통량이 -1인 행은 의미없는 값으로 제외

In [9]:
df = df[df['교통량'] != -1]

In [10]:
df['평균속도'].describe()

count    3.600852e+07
mean     7.404197e+01
std      4.283381e+01
min      0.000000e+00
25%      6.400000e+01
50%      8.900000e+01
75%      1.025000e+02
max      1.800000e+02
Name: 평균속도, dtype: float64

In [11]:
df['점유율'].describe()

count    3.600852e+07
mean     3.971261e+00
std      5.963955e+00
min      0.000000e+00
25%      6.100000e-01
50%      2.310000e+00
75%      5.350000e+00
max      1.000000e+02
Name: 점유율, dtype: float64

## 30초 단위 데이터를 5분 단위 데이터로 취합

### 1. 평균속도에 대한 30초 단위 데이터 aggregation

- [국가법령정보센터 차량검지기(VDS) 성능평가 기준 1. 개요 가. 대상장비](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwi-xKiq-5CBAxUxh1YBHZlNA8gQFnoECBAQAQ&url=https%3A%2F%2Fwww.law.go.kr%2FflDownload.do%3FflSeq%3D72668643%26flNm%3D%255B%25EB%25B3%2584%25ED%2591%259C%2B4%255D%2B%25EC%25B0%25A8%25EB%259F%2589%25EA%25B2%2580%25EC%25A7%2580%25EA%25B8%25B0%2528VDS%2529%2B%25EC%2584%25B1%25EB%258A%25A5%25ED%258F%2589%25EA%25B0%2580%2B%25EA%25B8%25B0%25EC%25A4%2580&usg=AOvVaw0JvMC9hj7b-wJFfJSvLYjh&opi=89978449)
  - 교통량: 분석단위시간당 측정된 차량의 합
  - 속도: 분석단위시간당 측정된 차량의 속도를 산술평균한 값
  - 여기서 각 행마다 속도와 교통량을 곱하면 분석단위시간당 측정된 차량의 속도의 누적합을 구할 수 있으며, 30초 단위 데이터마다 이를 수행하여 5분 분석단위시간당 측정된 차량의 속도 누적합을 구함
  - 이렇게 한 누적합된 속도를 5분 분석단위시간동안 측정된 차량의 합으로 나누어, 5분 동안에 측정된 차량의 속도를 산술평균한 값을 구할 수 있음

In [12]:
def aggregate_speed(data: pd.DataFrame):
    if len(data) == 0:
        return -1

    speed_sum = np.inner(data['교통량'], data['평균속도']) # 각 데이터에 대한 속도의 누적합을 구한다.
    count = sum(data['교통량']) # 교통량의 합을 구해 검지기를 지난 차량의 수를 구한다.

    if count == 0:
        return 0

    return speed_sum / count

### 2. 점유율에 대한 30초 단위 데이터 aggregation
- [국가법령정보센터 차량검지기(VDS) 성능평가 기준 1. 개요 가. 대상장비](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwi-xKiq-5CBAxUxh1YBHZlNA8gQFnoECBAQAQ&url=https%3A%2F%2Fwww.law.go.kr%2FflDownload.do%3FflSeq%3D72668643%26flNm%3D%255B%25EB%25B3%2584%25ED%2591%259C%2B4%255D%2B%25EC%25B0%25A8%25EB%259F%2589%25EA%25B2%2580%25EC%25A7%2580%25EA%25B8%25B0%2528VDS%2529%2B%25EC%2584%25B1%25EB%258A%25A5%25ED%258F%2589%25EA%25B0%2580%2B%25EA%25B8%25B0%25EC%25A4%2580&usg=AOvVaw0JvMC9hj7b-wJFfJSvLYjh&opi=89978449)
  - “점유시간”(초)이라 함은 VDS 기본성능평가에서 차량이 검지영역을 통과하는데 소요되는 시간을 말한다.
  - “점유율”(%)이라 함은 VDS 기본성능평가에서 분석 단위시간당 측정된 점유시간의 합을 백분율로 나타낸 것을 말한다.
- 결국, 점유율은 VDS를 운영하는 단위 시간(여기서는 30초) 동안 차량들이 그 위(VDS)를 얼마 시간동안 올라가 있었는지를 말한다.
- 이를 통해, 30초 동안의 점유 시간을 계산할 수 있고, 이를 모두 합하여 5분 동안의 점유율을 구할 수 있다.

In [13]:
def aggregate_share(data: pd.DataFrame):
    if len(data) == 0:
        return -1

    time_sum = sum(data['점유율'] / 100 * 30) # 백분율을 비율로 환산, 30초 곱하여 단위시간당 측정된 점유시간의 합 계산
    time_total = 300 # 5분 == 300초

    return time_sum / time_total * 100

### 3. 교통량에 대한 30초 단위 데이터 aggregation

In [14]:
def aggregate_traffic(data: pd.DataFrame):
    if len(data) == 0:
        return -1

    return sum(data['교통량']) # 단순 누적합

## 우리 DB에서 사용하는 5분 데이터와 값 비교

In [15]:
sample_df = pd.read_csv('vds_sample.csv', encoding='euc-kr', header=None, names=[
    '도로번호',
    '도로명',
    '콘존ID',
    '구간명',
    '구간길이(m)',
    '기점종점방향구분코드',
    '집계일자',
    '집계시분',
    'VDS_ID',
    '차로유형구분코드',
    '교통량',
    '점유율',
    '평균속도',
    'Unknown',
], dtype={
    '집계시분': str
})
sample_df

,도로번호,도로명,콘존ID,구간명,구간길이(m),기점종점방향구분코드,집계일자,집계시분,VDS_ID,차로유형구분코드,교통량,점유율,평균속도,Unknown
0,450,중부내륙선,0450CZS200,충주IC-괴산IC,14910.0,S,20220701,0115,0450VDS17800,1,36,2.76,95.38,NaN
1,450,중부내륙선,0450CZS200,충주IC-괴산IC,14910.0,S,20220701,0115,0450VDS17900,1,38,2.71,95.18,NaN
2,450,중부내륙선,0450CZS200,충주IC-괴산IC,14910.0,S,20220701,0115,0450VDS17951,1,35,2.99,93.63,NaN
3,450,중부내륙선,0450CZS200,충주IC-괴산IC,14910.0,S,20220701,0115,0450VDS18000,1,37,2.97,90.88,NaN
4,450,중부내륙선,0450CZS200,충주IC-괴산IC,14910.0,S,20220701,0115,0450VDS18100,1,36,2.87,93.91,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,500,영동선,0500CZE080,동군포IC-부곡IC,2150.0,E,20220701,0115,0500VDE02200,1,-1,-1.00,-1.00,NaN
96,500,영동선,0500CZE080,동군포IC-부곡IC,2150.0,E,20220701,0115,0500VDE02300,1,48,2.87,89.46,NaN
97,500,영동선,0500CZE090,부곡IC-북수원IC,2840.0,E,20220701,0115,0500VDE02400,1,-1,-1.00,-1.00,NaN
98,500,영동선,0500CZE090,부곡IC-북수원IC,2840.0,E,20220701,0115,0500VDE02500,1,54,3.27,87.73,NaN


In [16]:
from datetime import datetime, timedelta

result = []

for idx, row in tqdm(sample_df.iterrows()):
    item = []

    vds_id = row['VDS_ID']
    item.append(vds_id) # VDS_ID

    _start = datetime.strptime(row['집계시분'] + '00', '%H%M%S')
    _end = _start + timedelta(minutes=5)

    start = int(_start.strftime('%H%M%S'))
    end = int(_end.strftime('%H%M%S'))

    temp_df = df.query(f"VDS_ID == '{vds_id}' and 수집시분초 >= {start} and 수집시분초 < {end}")
    
    item.append(aggregate_traffic(temp_df)) # 취합교통량
    item.append(row['교통량']) # 5분교통량

    item.append(aggregate_share(temp_df)) # 취합점유율
    item.append(row['점유율']) # 5분점유율

    item.append(aggregate_speed(temp_df)) # 취합평균속도
    item.append(row['평균속도']) # 5분평균속도

    result.append(item)

result_df = pd.DataFrame(result, columns=['VDS_ID', '취합교통량', '5분교통량', '취합점유율', '5분점유율', '취합평균속도', '5분평균속도'])
result_df

1it [00:00,  1.45it/s]

100it [00:52,  1.90it/s]


,VDS_ID,취합교통량,5분교통량,취합점유율,5분점유율,취합평균속도,5분평균속도
0,0450VDS17800,36.0,36,4.897,2.76,97.472222,95.38
1,0450VDS17900,38.0,38,4.919,2.71,96.815789,95.18
2,0450VDS17951,35.0,35,5.418,2.99,95.028571,93.63
3,0450VDS18000,37.0,37,5.181,2.97,91.513514,90.88
4,0450VDS18100,36.0,36,5.175,2.87,95.138889,93.91
...,...,...,...,...,...,...,...
95,0500VDE02200,-1.0,-1,-1.000,-1.00,-1.000000,-1.00
96,0500VDE02300,48.0,48,7.360,2.87,89.520833,89.46
97,0500VDE02400,-1.0,-1,-1.000,-1.00,-1.000000,-1.00
98,0500VDE02500,54.0,54,7.548,3.27,87.907407,87.73


In [17]:
result_df.to_csv('vds_aggregation_result.csv', index=False)